In [1]:
import numpy as np
import pandas as pd
import sklearn
import torch

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [2]:
batch_size=1
seq_len=3
input_size=4
hidden_size=2

In [6]:
cell = torch.nn.RNNCell(input_size=input_size,hidden_size=hidden_size)
dataset=torch.randn(seq_len,batch_size,input_size)
hidden=torch.zeros(batch_size,hidden_size)

for idx, input  in enumerate(dataset):
    print('='*20,idx,'='*20)
    print('input size:',input.shape)
    hidden=cell(input,hidden)
    print('outputs size:',hidden.shape)
    print(hidden)

==================== 0 ====================
input size: torch.Size([1, 4])
outputs size: torch.Size([1, 2])
tensor([[ 0.0964, -0.0458]], grad_fn=<TanhBackward>)
==================== 1 ====================
input size: torch.Size([1, 4])
outputs size: torch.Size([1, 2])
tensor([[-0.7504, -0.8917]], grad_fn=<TanhBackward>)
==================== 2 ====================
input size: torch.Size([1, 4])
outputs size: torch.Size([1, 2])
tensor([[-0.0356, -0.7497]], grad_fn=<TanhBackward>)


In [8]:
batch_size=1
seq_len=3
input_size=4
hidden_size=2
num_layers=1

In [10]:
cell=torch.nn.RNN(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers)

inputs= torch.randn(seq_len,batch_size,input_size)
hidden=torch.zeros(num_layers,batch_size,hidden_size)

out,hidden=cell(inputs,hidden)

print("output size:",out.shape)
print("output:",out)

print("hidden size:",hidden.shape)
print("hidden:",hidden)



output size: torch.Size([3, 1, 2])
output: tensor([[[ 0.8923, -0.8049]],

        [[-0.2057, -0.9341]],

        [[-0.4651,  0.2450]]], grad_fn=<StackBackward>)
hidden size: torch.Size([1, 1, 2])
hidden: tensor([[[-0.4651,  0.2450]]], grad_fn=<StackBackward>)


# dmeo

In [11]:
batch_size=1
input_size=4
hidden_size=2

In [13]:
idx2char=['e','h','l','o']
x_data=[1,0,2,2,3]
y_data=[3,1,2,3,2]

one_hot_lookup=[
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1]
]

x_one_hot=[one_hot_lookup[x] for x in x_data]
inputs=torch.Tensor(x_one_hot).view(-1,batch_size,input_size)
labels=torch.LongTensor(y_data).view(-1,1)

In [15]:
class Model(torch.nn.Module):
    def __init__(self,input_size,hidden_size,batch_size):
        super(Model,self).__init__()
        self.batch_size=batch_size
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.rnncell=torch.nn.RNNCell(input_size=self.input_size,hidden_size=self.hidden_size)
        
    def forward(self,input,hidden):
        hidden=self.rnncell(input,hidden)
        return hidden
    
    def init_hidden(self):
        return torch.zeros(self.batch_size,self.hidden_size)
    


In [19]:
net = Model(input_size,hidden_size,batch_size)
criterion=torch.nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),lr=0.01)

In [23]:
for epoch in range(15):
    loss=0
    optimizer.zero_grad()
    hidden=net.init_hidden()
    print('predicted string:',end='')
    for input, label in zip(inputs,labels):
        hidden=net(input,hidden)
        loss+=criterion(hidden,label)
        _,idx=hidden.max(dim=1)
        print(idx2char[idx.item()],end='')
        
    loss.backward()
    optimizer.step()
    print(',epoch[%d/15] loss=%.4f' % (epoch+1,loss.item()))
        

predicted string:

IndexError: Target 3 is out of bounds.